In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)


In [2]:
df = pd.read_csv("ovb.csv")
df['const'] = 1

In [3]:
#Collecting women in the data:
dfwomen = df[df.female == 1]

In [4]:
#Correlation Coefficient:
corr_sq = np.corrcoef(dfwomen.educ, dfwomen.logwage)[1][0]**2

print("The correlation coefficient squared between education and logwage of women is: {}".format(corr_sq))

The correlation coefficient squared between education and logwage of women is: 0.22388700299289635


In [5]:
reg = sm.OLS(dfwomen.logwage, dfwomen[['educ', 'const']]).fit()


In [6]:
sumtable = np.vstack([[reg.rsquared], [corr_sq]])
sumtable = np.round(sumtable, 6)
col = [' R**2 vs Corr Coef' ]
row = ['Rsquared','corrsquared' ]
tb1 = SimpleTable(sumtable,col, row, txt_fmt=default_txt_fmt)

In [7]:
print(tb1)

             R**2 vs Corr Coef
------------------------------
Rsquared         0.223887     
corrsquared      0.223887     
------------------------------


### 3

#### a) 
The unpooled t-test shows that the mean difference between female immigrants and female non-immigrant logwages are significant. 
        t-test = -1111.506

In [18]:
#Mean and standard error imm = 0: X
Mx = dfwomen.loc[dfwomen['imm'] == 0].logwage.mean()
nx = dfwomen.loc[dfwomen['imm'] == 0].logwage.count()
sex = np.std(dfwomen.loc[dfwomen['imm'] == 0].logwage) / np.sqrt(nx)

print("The mean of female non-immigrants: {}".format(Mx))
print("The standard error of female non-immigrants: {}".format(sex))


The mean of female non-immigrants: 2.886378280700209
The standard error of female non-immigrants: 0.007018633901173876


In [19]:
#Mean and standard error imm = 1: Y
My = dfwomen.loc[dfwomen['imm'] == 1].logwage.mean()
ny = dfwomen.loc[dfwomen['imm'] == 1].logwage.count()
sey = np.std(dfwomen.loc[dfwomen['imm'] == 1].logwage) / np.sqrt(ny)
print("The mean of female immigrants: {}".format(My))
print("The standard error of female immigrants: {}".format(sey))



The mean of female immigrants: 2.706392526468514
The standard error of female immigrants: 0.016061501364901087


##### Test statistic:

In [22]:
ttest = (My-Mx) / np.sqrt((sex**2 + sey**2))
print("The unpooled t-test for the difference: {}".format(ttest))
print('Difference in mean: {}'.format(My-Mx))



The unpooled t-test for the difference: -10.268432855577307
Difference in mean: -0.17998575423169516


From this result we can see that we may reject the null hypothesis, that the means are the same at 5% support.

#### b)

In [24]:
reg1 = sm.OLS(dfwomen.logwage, dfwomen[['const','imm']]).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logwage   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     118.5
Date:                Thu, 28 Feb 2019   Prob (F-statistic):           1.85e-27
Time:                        10:18:58   Log-Likelihood:                -10701.
No. Observations:               10601   AIC:                         2.141e+04
Df Residuals:                   10599   BIC:                         2.142e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.8864      0.007    403.480      0.000       2.872       2.900
imm           -0.1800      0.017    -10.887      0.000      -0.212      -0.148
==============================================================================
Omnibus:                      353.044   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              538.131
Skew:                           0.324   Prob(JB):                    1.40e-117
Kurtosis:                       3.893   Cond. No.                         2.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
beta_imm = reg1.params[1]
print("The mean difference for the mean and the immigration coefficient from OLS imm is : {}".format((My-Mx) - beta_imm))




The mean difference for the mean and the immigration coefficient from OLS imm is : 9.71445146547012e-16


In [26]:
#Test statistic for beta:
sem1 = reg1.bse[1]
t_beta = beta_imm / sem1
t_beta

-10.887143729424317

The t-tests conducted are seemingly different. This is intuitive since python's OLS assumes homoskedasticity. In this case when the error are changing across covariates, assuming homoskedasticity is not optimal.

In [27]:
se = np.vstack([[reg1.bse], [reg1.HC1_se]])
se = np.round(se,5)
col = ['constant', 'Immigrant']
row = ['OLS', "Heteroskedasticity-robust"]
tb1 = SimpleTable(se,col, row, txt_fmt=default_txt_fmt)
tb1

,constant,Immigrant
OLS,0.00715,0.01653
Heteroskedasticity-robust,0.00702,0.01753


##### C)

In [32]:
covHC = reg1.cov_HC1
std = np.sqrt(covHC.diagonal())
covHC

array([[ 4.92705173e-05, -4.92705173e-05],
       [-4.92705173e-05,  3.07291022e-04]])

Taking the square-root of the diagonal ( diagonal has SE^2 for beta 0 and beta 1):


In [33]:
std

array([0.0070193 , 0.01752972])

These values are slightly different from what was found above, and I assume it to be that I may be using a different method for retaining HC variance-cov